In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from prettytable import PrettyTable


In [3]:
def avg_diff(values):
    # return the average pairwise different between all the pairs in the list
    pairs = [(a, b) for idx, a in enumerate(values) for b in values[idx + 1:]]
    difference = list(map(lambda x: abs(x[0] - x[1]), pairs))
    return sum(difference)/len(difference)


In [1]:
# score_dir = '../bold/result/score'
# score_dir = 'results/gpt2-medium-top_p=09/score'
# score_dir = 'results/dexperts_gpt2_antionly_alpha2/score'
score_dir = 'results/dexperts_gpt2_med_alpha1/score/'
# score_dir = 'results/dexperts_gpt2_antionly_alpha2/score'
path_to_dir = os.path.relpath(score_dir)
# get the json files in the directory
csv_files = [pos_csv for pos_csv in os.listdir(path_to_dir) if pos_csv.endswith('.csv')]

df = pd.concat([pd.read_csv(os.path.join(path_to_dir, file))
               for file in csv_files])

undesired_subgroups = ['Asian_Americans', 'Hispanic_and_Latino_Americans']
df = df[~df['subgroup'].isin(undesired_subgroups)]

In [2]:
df

,model,group,subgroup,metric,score
5,dexperts_gpt2-medium,gpt2-medium_race,African_Americans,regard-positive,0.4712
6,dexperts_gpt2-medium,gpt2-medium_race,African_Americans,regard-negative,0.1170
7,dexperts_gpt2-medium,gpt2-medium_race,African_Americans,regard-neutral,0.3662
8,dexperts_gpt2-medium,gpt2-medium_race,African_Americans,regard-other,0.0455
9,dexperts_gpt2-medium,gpt2-medium_race,African_Americans,toxicity-ratio,0.0076
10,dexperts_gpt2-medium,gpt2-medium_race,European_Americans,regard-positive,0.5036
11,dexperts_gpt2-medium,gpt2-medium_race,European_Americans,regard-negative,0.0844
12,dexperts_gpt2-medium,gpt2-medium_race,European_Americans,regard-neutral,0.3690
13,dexperts_gpt2-medium,gpt2-medium_race,European_Americans,regard-other,0.0429
14,dexperts_gpt2-medium,gpt2-medium_race,European_Americans,toxicity-ratio,0.0053


In [30]:
regardTable = PrettyTable(
    ["Domain", "Model", "Positive", "Neutral", "Negative"])
toxicityTable = PrettyTable(["Domain", "Model", "Ratio"])

In [31]:
scores = []
groups = df['group'].unique().tolist()

for group in groups:
    # group = 'political_ideology'
    data = df[df['group'] == group]
    model_name = data[data['metric'] ==
                      'regard-positive']['model'].values.tolist()
    subgroup = data[data['metric'] ==
                    'regard-positive']['subgroup'].values.tolist()
    labels = [model_name[i] + '\n' + subgroup[i]
              for i in range(len(model_name))]

    model_name = data[data['metric'] ==
                      'regard-positive']['model'].unique().tolist()
    subgroup = data[data['metric'] ==
                    'regard-positive']['subgroup'].unique().tolist()

    positive_regards = np.array(
        data[data['metric'] == 'regard-positive']['score'].values.tolist())
    negative_regards = np.array(
        data[data['metric'] == 'regard-negative']['score'].values.tolist())
    neutral_regards = np.array(
        data[data['metric'] == 'regard-neutral']['score'].values.tolist())
    toxicity = np.array(
        data[data['metric'] == 'toxicity-ratio']['score'].values.tolist())

    n_subgroups = len(subgroup)
    for i in range(len(model_name)):
        start_ind, end_ind = n_subgroups * i, n_subgroups * (i+1)
        positive = positive_regards[start_ind:end_ind]
        negative = negative_regards[start_ind:end_ind]
        neutral = neutral_regards[start_ind:end_ind]
        toxic = toxicity[start_ind:end_ind]

        regardTable.add_row([group, model_name[i], round(avg_diff(positive), 4), round(
            avg_diff(neutral), 4), round(avg_diff(negative), 4)])
        toxicityTable.add_row(
            [group, model_name[i], round(avg_diff(toxic), 4)])

        scores.append({'model': model_name[i], 'group': group,  'positive': round(avg_diff(positive), 4),
                       'negative': round(avg_diff(negative), 4), 'neutral': round(avg_diff(neutral), 4), 'toxicity_ratio': round(avg_diff(toxic), 4)})


In [21]:
regardTable

Domain,Model,Positive,Neutral,Negative
gpt2-medium_race,dexperts_gpt2-medium,0.0388,0.0184,0.0253
gpt2-medium_gender,dexperts_gpt2-medium,0.0118,0.0134,0.0048
gpt2-medium_religious_ideology,dexperts_gpt2-medium,0.1297,0.0824,0.1452


In [17]:
regardTable.get_csv_string()

'Domain,Model,Positive,Neutral,Negative\r\ngpt2-medium_race,dexperts_gpt2-medium,0.0388,0.0184,0.0253\r\ngpt2-medium_gender,dexperts_gpt2-medium,0.0118,0.0134,0.0048\r\ngpt2-medium_religious_ideology,dexperts_gpt2-medium,0.1297,0.0824,0.1452\r\n'

In [15]:
pd.DataFrame()

ValueError: DataFrame constructor not properly called!

In [99]:
toxicityTable

Domain,Model,Ratio
gpt2-medium_race,dexperts_gpt2-medium,0.0023
gpt2-medium_gender,dexperts_gpt2-medium,0.0027
gpt2-medium_religious_ideology,dexperts_gpt2-medium,0.0645


In [25]:
#save to csv
df = pd.DataFrame.from_dict(scores)
df.to_csv (r'relative_regard_scores.csv', index = False, header=True)